In [59]:
import numpy as np
from lab3_tools import *
from lab3_proto import *
from lab2_tools import *
from lab2_proto import *
from lab1_proto import *
from lab1_tools import *
import torch

from sklearn.preprocessing import StandardScaler

In [14]:
phoneHMMs = np.load('lab2_models_all.npz', allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]
stateList

['ah_0',
 'ah_1',
 'ah_2',
 'ao_0',
 'ao_1',
 'ao_2',
 'ay_0',
 'ay_1',
 'ay_2',
 'eh_0',
 'eh_1',
 'eh_2',
 'ey_0',
 'ey_1',
 'ey_2',
 'f_0',
 'f_1',
 'f_2',
 'ih_0',
 'ih_1',
 'ih_2',
 'iy_0',
 'iy_1',
 'iy_2',
 'k_0',
 'k_1',
 'k_2',
 'n_0',
 'n_1',
 'n_2',
 'ow_0',
 'ow_1',
 'ow_2',
 'r_0',
 'r_1',
 'r_2',
 's_0',
 's_1',
 's_2',
 'sil_0',
 'sil_1',
 'sil_2',
 'sp_0',
 't_0',
 't_1',
 't_2',
 'th_0',
 'th_1',
 'th_2',
 'uw_0',
 'uw_1',
 'uw_2',
 'v_0',
 'v_1',
 'v_2',
 'w_0',
 'w_1',
 'w_2',
 'z_0',
 'z_1',
 'z_2']

In [15]:
np.savez("./statelist.npz",stateList)

In [16]:
test_example, _= loadAudio('tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav')
path2info('tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav')

('man', 'ae', 'z9z6531', 'a')

In [17]:
filename = 'tidigits/disc_4.1.1/tidigits/train/man/nw/z43a.wav'
samples, samplingrate = loadAudio(filename)
lmfcc = mfcc(samples)
wordTrans = list(path2info(filename)[2])
wordTrans

['z', '4', '3']

In [6]:
from prondict import prondict
phoneTrans = words2phones(wordTrans, prondict)
phoneTrans

['sil',
 'z',
 'iy',
 'r',
 'ow',
 'sp',
 'f',
 'ao',
 'r',
 'sp',
 'th',
 'r',
 'iy',
 'sil']

In [7]:
utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)

In [8]:
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
                  for stateid in range(nstates[phone])]
stateTrans[10]

'r_1'

In [9]:
viterbiStateTrans = forcedAlignment(lmfcc, phoneHMMs, phoneTrans)

/Users/cubeddu_laflame/Desktop/speech-and-speaker-recognition-2/lab3/lab3_proto.py:66: RuntimeWarning: divide by zero encountered in log
  _, viterbi_path = viterbi(observation_log_likelihood, np.log(utterance_HMM['startprob']), np.log(utterance_HMM['transmat']), forceFinalState=True)


In [10]:
frames2trans(viterbiStateTrans, outfilename='z43a.lab')

'0 0.01 sil_0\n0.01 0.19000000000000003 sil_1\n0.19000000000000003 0.20000000000000004 sil_2\n0.20000000000000004 0.24000000000000007 z_0\n0.24000000000000007 0.25000000000000006 z_1\n0.25000000000000006 0.36000000000000015 z_2\n0.36000000000000015 0.4400000000000002 iy_0\n0.4400000000000002 0.45000000000000023 iy_1\n0.45000000000000023 0.46000000000000024 iy_2\n0.46000000000000024 0.5600000000000003 r_0\n0.5600000000000003 0.5700000000000003 r_1\n0.5700000000000003 0.5800000000000003 r_2\n0.5800000000000003 0.5900000000000003 ow_0\n0.5900000000000003 0.6000000000000003 ow_1\n0.6000000000000003 0.6900000000000004 ow_2\n0.6900000000000004 0.7000000000000004 sp_0\n0.7000000000000004 0.7100000000000004 f_0\n0.7100000000000004 0.8100000000000005 f_1\n0.8100000000000005 0.8200000000000005 f_2\n0.8200000000000005 0.8300000000000005 ao_0\n0.8300000000000005 0.9700000000000006 ao_1\n0.9700000000000006 1.0800000000000007 ao_2\n1.0800000000000007 1.1100000000000008 r_0\n1.1100000000000008 1.1200

In [11]:
import os
traindata = []
for root, dirs, files in os.walk('tidigits/disc_4.1.1/tidigits/train'):
    for file in files:
      if file.endswith('.wav'):
          filename = os.path.join(root, file)
          samples, samplingrate = loadAudio(filename)
          lmfcc = mfcc(samples)
          spec = mspec(samples)
          wordTrans = list(path2info(filename)[2])
          phoneTrans = words2phones(wordTrans, prondict)
          targets = forcedAlignment(lmfcc, phoneHMMs, phoneTrans)
          traindata.append({'filename': filename, 'lmfcc': lmfcc, 'mspec': 'mspec', 'targets': targets})

KeyboardInterrupt: 

In [ ]:
def loaddata():
    traindata = []
    for root, dirs, files in os.walk():
        for file in files:
          if file.endswith('.wav'):
              filename = os.path.join(root, file)
              samples, samplingrate = loadAudio(filename)
              lmfcc = mfcc(samples)
              spec = mspec(samples)
              wordTrans = list(path2info(filename)[2])
              phoneTrans = words2phones(wordTrans, prondict)
              targets = forcedAlignment(lmfcc, phoneHMMs, phoneTrans)
              traindata.append({'filename': filename, 'lmfcc': lmfcc, 'mspec': 'mspec', 'targets': targets})
    return traindata

In [ ]:
traindata = loaddata('tidigits/disc_4.1.1/tidigits/train')
np.savez('traindata.npz', traindata=traindata)

TypeError: loaddata() takes 0 positional arguments but 1 was given

In [ ]:
testdata = loaddata('tidigits/disc_4.1.1/tidigits/test')
np.savez('testdata.npz', testdata=testdata)

TypeError: loaddata() takes 0 positional arguments but 1 was given

In [45]:
train_data = np.load('traindata.npz', allow_pickle=True)['traindata']
n_val = round(len(train_data)//10)
testdata=np.load('testdata.npz', allow_pickle=True)['testdata']

1507392


In [31]:
indexes = np.random.permutation(len(train_data))
train = np.take(train_data,indexes)
val = train[:n_val]
train = train[n_val:]
np.savez('train.npz', train=train)
np.savez('val.npz', val=val)


NameError: name 'data' is not defined

## Acoustic Context (Dynamic Features)

In [46]:
def get_features(data, dynamic=True):
    lmfcc_dim = data[0]['lmfcc'].shape[1]
    mspec_dim = data[0]['mspec'].shape[1]
    total_frames = sum([len(x['targets']) for x in data])
    print(total_frames)
    pad_size = 3
    if dynamic:
        num_features = 7
    else:
        num_features = 1
    dynamic2 = dynamic
    dynamic = False
    mfcc_features = np.zeros((total_frames, lmfcc_dim * num_features))
    mspec_features = np.zeros((total_frames, mspec_dim * num_features))
    targets = []
    current_frame_idx = 0 
    for i, utterance in enumerate(data):
        lmfcc_padded = np.pad(utterance['lmfcc'], pad_width=((pad_size, pad_size), (0, 0)), mode='reflect')
        mspec_padded = np.pad(utterance['mspec'], pad_width=((pad_size, pad_size), (0, 0)), mode='reflect')
        num_frames = len(utterance['targets'])
        for frame_idx in range(num_frames):
            if dynamic2:
                start = frame_idx
                end = frame_idx + num_features
                if start < pad_size:
                    start = pad_size
                    end = pad_size + num_features
                elif end > num_frames - pad_size:
                    start = num_frames - pad_size - num_features
                    end = num_frames - pad_size
                mfcc_features[current_frame_idx] = np.hstack(lmfcc_padded[start:end])
                mspec_features[current_frame_idx] = np.hstack(mspec_padded[start:end])
            else:
                mfcc_features[current_frame_idx] = utterance['lmfcc'][frame_idx]
                mspec_features[current_frame_idx] = utterance['mspec'][frame_idx]

            if frame_idx < len(utterance['targets']):
                targets.append(utterance['targets'][frame_idx])
            else:
                # Handle case when 'targets' list is shorter than expected
                targets.append(None)   
            current_frame_idx += 1
    
    return mfcc_features, mspec_features, targets


In [47]:
d_lmfcc_train, d_mspec_train, train_y = get_features(train,dynamic=True)
d_lmfcc_val, d_mspec_val, val_y = get_features(val,dynamic=True)
d_lmfcc_test, d_mspec_test, test_y = get_features(testdata,dynamic=True)

1354824
152568
1527014


In [48]:
lmfcc_train_x, mspec_train_x, _ = get_features(train,dynamic=False)
lmfcc_val_x, mspec_val_x, _ = get_features(val,dynamic=False)
lmfcc_test_x, mspec_test_x, _ = get_features(testdata,dynamic=False)


1354824
152568
1527014


In [49]:
np.savez('d_lmfcc_train.npz', d_lmfcc_train=d_lmfcc_train)
np.savez('d_lmfcc_val.npz', d_lmfcc_val=d_lmfcc_val)
np.savez('d_lmfcc_test.npz', d_lmfcc_test=d_lmfcc_test)

np.savez('d_mspec_train.npz', d_mspec_train=d_mspec_train)
np.savez('d_mspec_val.npz', d_mspec_val=d_mspec_val)
np.savez('d_mspec_test.npz', d_mspec_test=d_mspec_test)

np.savez('train_y',train_y=train_y)
np.savez('val_y',val_y=val_y)
np.savez('test_y',test_y=test_y)

In [50]:
np.savez('lmfcc_train_x.npz', lmfcc_train_x=lmfcc_train_x)
np.savez('lmfcc_val_x.npz', lmfcc_val_x=lmfcc_val_x)
np.savez('lmfcc_test_x.npz', lmfcc_test_x=lmfcc_test_x)
np.savez('mspec_train_x.npz', mspec_train_x=mspec_train_x)
np.savez('mspec_val_x.npz', mspec_val_x=mspec_val_x)
np.savez('mspec_test_x.npz', mspec_test_x=mspec_test_x)

## Feature Standardisation

In [51]:
stateList = np.load('statelist.npz',allow_pickle=True)['arr_0']
output_dim = len(stateList)

In [52]:
#You can use the StandardScaler from sklearn.preprocessing in order to achieve this. In case you normalise over the whole training set, save the normalisation coefficients and reuse them to normalise the validation and test set. In this case, it is also easier to perform the following step before standardisation
scaler = StandardScaler()
scaler.fit(d_lmfcc_train)
d_lmfcc_train = scaler.transform(d_lmfcc_train)
d_lmfcc_val = scaler.transform(d_lmfcc_val)
d_lmfcc_test = scaler.transform(d_lmfcc_test)
scaler2=StandardScaler()
scaler2.fit(d_mspec_train)
d_mspec_train = scaler2.transform(d_mspec_train)
d_mspec_val = scaler2.transform(d_mspec_val)
d_mspec_test = scaler2.transform(d_mspec_test)
scaler3=StandardScaler()
scaler3.fit(lmfcc_train_x)
lmfcc_train_x = scaler3.transform(lmfcc_train_x)
lmfcc_val_x = scaler3.transform(lmfcc_val_x)
lmfcc_test_x = scaler3.transform(lmfcc_test_x)
scaler4=StandardScaler()
scaler4.fit(mspec_train_x)
mspec_train_x = scaler4.transform(mspec_train_x)
mspec_val_x = scaler4.transform(mspec_val_x)
mspec_test_x = scaler4.transform(mspec_test_x)


In [77]:
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
lmfcc_train_x = lmfcc_train_x.astype('float32')
lmfcc_val_x = lmfcc_val_x.astype('float32')
lmfcc_test_x = lmfcc_test_x.astype('float32')
output_dim = len(stateList)

# Convert string labels to numerical format
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.fit_transform(val_y)
test_y_encoded = label_encoder.fit_transform(test_y)
# Apply one-hot encoding to the numerical labels
one_hot_train_y = F.one_hot(torch.tensor(train_y_encoded), num_classes=output_dim)
one_hot_val_y = F.one_hot(torch.tensor(val_y_encoded), num_classes=output_dim)
one_hot_test_y = F.one_hot(torch.tensor(test_y_encoded), num_classes=output_dim)





In [89]:
#reshape lmfcc_train_x removing the last dimension
lmfcc_train_x = lmfcc_train_x.reshape(lmfcc_train_x.shape[0],lmfcc_train_x.shape[1])




lmfcc_train_x shape:  (1354824, 13)
lmfcc_val_x shape:  (152568, 13)
lmfcc_test_x shape:  (1527014, 13)
mspec_train_x shape:  (1354824, 40)
mspec_val_x shape:  (152568, 40)
mspec_test_x shape:  (1527014, 40)
d_lmfcc_train shape:  (1354824, 91)
d_lmfcc_val shape:  (152568, 91)
d_lmfcc_test shape:  (1527014, 91)
d_mspec_train shape:  (1354824, 280)
d_mspec_val shape:  (152568, 280)
d_mspec_test shape:  (1527014, 280)


AttributeError: 'list' object has no attribute 'shape'

In [90]:
#save data
print(d_lmfcc_train.shape)
np.savez('d_lmfcc_train.npz', d_lmfcc_train=d_lmfcc_train)
np.savez('d_lmfcc_val.npz', d_lmfcc_val=d_lmfcc_val)
np.savez('d_lmfcc_test.npz', d_lmfcc_test=d_lmfcc_test)
np.savez('d_mspec_train.npz', d_mspec_train=d_mspec_train)
np.savez('d_mspec_val.npz', d_mspec_val=d_mspec_val)
np.savez('d_mspec_test.npz', d_mspec_test=d_mspec_test)
np.savez('one_hot_train_y.npz', one_hot_train_y=one_hot_train_y)
np.savez('one_hot_val_y.npz', one_hot_val_y=one_hot_val_y)
np.savez('one_hot_test_y.npz', one_hot_test_y=one_hot_test_y)
np.savez('lmfcc_train_x.npz', lmfcc_train_x=lmfcc_train_x)
np.savez('lmfcc_val_x.npz', lmfcc_val_x=lmfcc_val_x)
np.savez('lmfcc_test_x.npz', lmfcc_test_x=lmfcc_test_x)
np.savez('mspec_train_x.npz', mspec_train_x=mspec_train_x)
np.savez('mspec_val_x.npz', mspec_val_x=mspec_val_x)
np.savez('mspec_test_x.npz', mspec_test_x=mspec_test_x)



(1354824, 91)
